# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kuqa,41.7278,82.9364,-3.48,68,3,1.49,CN,2025-1-9
1,1,namibe,-15.1961,12.1522,22.91,86,7,1.22,AO,2025-1-9
2,2,opuwo,-18.0607,13.8400,23.38,46,87,1.37,NaN,2025-1-9
3,3,puerto ayora,-0.7393,-90.3518,27.34,96,79,0.89,EC,2025-1-9
4,4,wenchang,30.9040,118.4887,7.59,23,0,2.00,CN,2025-1-9


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
mapa = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles = True, alpha=0.70, marker='o', 
                                    s='Humidity', c='City', hover_alpha=1, frame_height=350)

# Display the map
mapa

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
cond1 = city_data_df['Max Temp'] <= 17
cond2 = city_data_df['Humidity'] <= 60
cond3 = city_data_df['Cloudiness'] >= 20.0
ideal_cities_df = city_data_df.where(cond1 & cond2 & cond3)

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18.0,dongning,44.0622,131.1207,-6.04,52.0,53.0,3.27,CN,2025-1-9
98,98.0,tripoli,32.8752,13.1875,16.66,52.0,74.0,4.65,LY,2025-1-9
122,122.0,alamosa,37.4695,-105.8700,-4.86,57.0,75.0,4.12,US,2025-1-9
162,162.0,hasaki,35.7333,140.8333,10.21,31.0,31.0,9.25,JP,2025-1-9
238,238.0,'alaqahdari dishu,30.4374,63.3130,10.49,52.0,97.0,2.72,AF,2025-1-9


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
18,dongning,CN,44.0622,131.1207,52.0,
98,tripoli,LY,32.8752,13.1875,52.0,
122,alamosa,US,37.4695,-105.8700,57.0,
162,hasaki,JP,35.7333,140.8333,31.0,
238,'alaqahdari dishu,AF,30.4374,63.3130,52.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    # print(name_address)

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    

# Display sample data
hotel_df

Starting hotel search
dongning - nearest hotel: Hepburn Hotel
tripoli - nearest hotel: الاصيل
alamosa - nearest hotel: 40 Winks Inn
hasaki - nearest hotel: Choshi Plaza Hotel
'alaqahdari dishu - nearest hotel: No hotel found
dalton - nearest hotel: Days Inn
zintan - nearest hotel: فندق الضيافة
sanming - nearest hotel: 三明客家宾馆
shingu - nearest hotel: ホテルニューパレス
yatou - nearest hotel: 盛悦商务酒店
filey - nearest hotel: Downcliffe House Hotel
den burg - nearest hotel: De Lindeboom
khost - nearest hotel: No hotel found
birmingham - nearest hotel: The Redmont
guelmim - nearest hotel: فندق الملتقى
zabol - nearest hotel: No hotel found
dalandzadgad - nearest hotel: Хан Уул буудал
livadiya - nearest hotel: Голд бич
colchani - nearest hotel: Hotel de Sal - Sol de Mañana
lorgues - nearest hotel: Hôtel du Parc


,City,Country,Lat,Lng,Humidity,Hotel Name
18,dongning,CN,44.0622,131.1207,52.0,Hepburn Hotel
98,tripoli,LY,32.8752,13.1875,52.0,الاصيل
122,alamosa,US,37.4695,-105.8700,57.0,40 Winks Inn
162,hasaki,JP,35.7333,140.8333,31.0,Choshi Plaza Hotel
238,'alaqahdari dishu,AF,30.4374,63.3130,52.0,No hotel found
245,dalton,US,34.7698,-84.9702,60.0,Days Inn
262,zintan,LY,31.9223,12.2520,45.0,فندق الضيافة
272,sanming,CN,26.2486,117.6186,56.0,三明客家宾馆
338,shingu,JP,33.7333,135.9833,44.0,ホテルニューパレス
343,yatou,CN,37.1500,122.3833,57.0,盛悦商务酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
mapb = hotel_df.hvplot.points('Lng', 'Lat', 
    geo=True, tiles = True, alpha=0.70, marker='o', 
    frame_width = 700, frame_height = 500,
    s='Humidity', c='City', 
    hover_alpha=1, 
    hover_cols=['Hotel Name', 'Country']
)
# Display the map
mapb

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)